In [14]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
from UrlParserForKBO import UrlParserForKBO

import numpy as np
import pandas as pd
import urllib2

In [15]:
#URL넣어주면 해당 박스스코어 파싱
class Parser_KBO:
    def __init__(self,url):
        data = urllib2.urlopen(url)
        html = BeautifulSoup(data)
        self.boxScore={}
        
        self.boxScore['url'=url]
        #DATE
        self.boxScore[date] = html.select_one('div.yearDate span').text.replace('.','')
        
        #AWAY팀명
        self.awayTeam=html.select_one('table.socreBoard tr:nth-of-type(2) th').text
        
        #HOME팀명
        self.homeTeam=html.select_one('table.socreBoard tr:nth-of-type(3) th').text
        
        #AWAY_SCORE
        tmp=html.select('table.socreBoard tr:nth-of-type(2) td')
        self.awayScore=list()
        for score in tmp:
            if score.text==u'-':
                self.awayScore.append(0)
            else:
                self.awayScore.append(int(score.text))
        
        #HOME_SCORE
        tmp=html.select('table.socreBoard tr:nth-of-type(3) td')
        self.homeScore=list()
        for score in tmp:
            if score.text==u'-':
                self.homeScore.append(0)
            else:
                self.homeScore.append(int(score.text))
        
        #AWAY_SUM_OF_SCORE
        self.awayScoreSum = self.awayscore[12]
        #HOME_SUM_OF_SCORE
        self.homeScoreSum = self.homescore[12]
        
        #AWAY_STATUS
        self.awayStatus=html.select_one('div.left p.results').text
        status=html.select_one('div.left p.results').text
        self.awayWinCount=status[:status.find(u'승')]
        self.awayLoseCount=status[status.find(u'승'):status.find(u'패')]
        self.awayDrawCount=status[status.find(u'패'):status.find(u'무')]
        
        #HOME_STATUS
        self.homeStatus=html.select_one('div.left p.results').text
        status=html.select_one('div.left p.results').text
        self.homeWinCount=status[:status.find(u'승')]
        self.homeLoseCount=status[status.find(u'승'):status.find(u'패')]
        self.homeDrawCount=status[status.find(u'패'):status.find(u'무')]
               
        #STADIUM
        tmp=html.select('p.ballpark')[0]
        #.text 메서드는 unicode형태로 반환
        tmp=tmp.text
        self.stadium=tmp[tmp.find(u'구장')+4:tmp.find(u'관중')].strip()
        
        #HEAD
        headRowNum = len(html.select('table.tEx tbody tr'))
        head=dict()
        for i in range(headRowNum):
            key = html.select('table.tEx tr:nth-of-type(%d) th'%(i+1))[0].text
            dat = html.select('table.tEx tr:nth-of-type(%d) td'%(i+1))[0].text
            dat = dat.strip()
            
            if key==u'심판':
                dat = dat.split()
            else:
                dat = dat.replace(') ',')) ').split(') ')
            head[key]=dat
        
        #AWAY 투수기록
        columns=[u'선수명',u'등판',u'결과',u'승',u'패',u'세',u'이닝',u'타자',u'투구수',u'타수',u'피안타',u'홈런',u'4사구',u'삼진',u'실점',u'자책',u'평균자책점']
        frame=html.select('table#xtable3 tbody:nth-of-type(1) tr')
        data=list()    #frame으로부터 데이터를 담을 리스트
        for row in frame:
            tmp=list()
            for dat in row.text.strip().split('\n'):    #한 행(한 투수의 데이터)마다 split한뒤 list로 정리
                tmp.append(dat)
            data.append(tmp)   #정리된 list를 data에 추가
        self.awayPitRecord = pd.DataFrame(data=data,columns=columns)

        #HOME 투수기록
        #col=[u'선수명',u'등판',u'결과',u'승',u'패',u'세',u'이닝',u'타자',u'투구수',u'타수',u'피안타',u'홈런',u'4사구',u'삼진',u'실점',u'자책',u'평균자책점']
        frame=html.select('table#xtable3 tbody:nth-of-type(2) tr')
        data=list()    #frame으로부터 데이터를 담을 리스트
        for row in frame:
            tmp=list()
            for dat in row.text.strip().split('\n'):    #한 행(한 투수의 데이터)마다 split한뒤 list로 정리
                tmp.append(dat)
            data.append(tmp)   #정리된 list를 data에 추가
        self.homePitRecord = pd.DataFrame(data=data,columns=columns)
        #등판:선발의 선수명과 이닝,타자를 get
        # -> homepitRecord[homepitRecord[u'등판']==u'선발'].reindex(columns=[u'선수명',u'이닝',u'타자'])
        
        #AWAY_타자기록
        columns=[u'선수명',u'1',u'2',u'3',u'4',u'5',u'6',u'7',u'8',u'9',u'10',u'11',u'12',u'타수',u'안타',u'타점',u'득점',u'타율']
        frame=html.select('table#xtable1 tbody:nth-of-type(1) tr')
        data=list()
        for row in frame:
            tmp=list()
            for dat in row.text.strip().split('\n')[2:]:
                tmp.append(dat)
            data.append(tmp)
        self.awayBatRecord = pd.DataFrame(data=data,columns=columns)
        
        #HOME_타자기록
        columns=[u'선수명',u'1',u'2',u'3',u'4',u'5',u'6',u'7',u'8',u'9',u'10',u'11',u'12',u'타수',u'안타',u'타점',u'득점',u'타율']
        frame=html.select('table#xtable1 tbody:nth-of-type(2) tr')
        data=list()
        for row in frame:
            tmp=list()
            for dat in row.text.strip().split('\n')[2:]:
                tmp.append(dat)
            data.append(tmp)
        self.homeBatRecord = pd.DataFrame(data=data,columns=columns)
        
        
        #승리팀,패배팀에 대한 멤버변수 설정
        if self.awayScore[12] > self.homeScore[12]:
            self.winTeam = self.awayteam
            self.winScore = self.awayscore[12]
            self.winTeamPitRecord = self.awayPitRecord
            self.winTeamBatRecord = self.awayBatRecord        
            self.winTeamStatus=self.awayStatus
            self.winTeamWinCount=self.awayWinCount
            self.winTeamLoseCount=self.awayLoseCount
            self.winTeamDrawCount=self.awayDrawCount
            
            self.loseTeam = self.homeTeam        
            self.loseScore = self.homeScore[12]
            self.loseTeamPitRecord = self.homePitRecord
            self.loseTeamBatRecord = self.homeBatRecord
            self.loseTeamStatus=self.homeStatus
            self.loseTeamWinCount=self.homeWinCount
            self.loseTeamLoseCount=self.homeLoseCount
            self.loseTeamDrawCount=self.homeDrawCount
            
        else:
            self.loseTeam= self.awayTeam
            self.loseScore= self.awayScore[12]
            self.loseTeampitRecord = self.awayPitRecord
            self.loseTeambatRecord = self.awayBatRecord        
            self.loseTeamStatus=self.awayStatus
            self.loseTeamWinCount=self.awayWinCount
            self.loseTeamLoseCount=self.awayLoseCount
            self.loseTeamDrawCount=self.awayDrawCount

            self.winTeam = self.homeTeam        
            self.winScore = self.homeScore[12]
            self.winTeamPitRecord = self.homePitRecord
            self.winTeamBatRecord = self.homeBatRecord
            self.winTeamStatus=self.homeStatus
            self.winTeamWinCount=self.homeWinCount
            self.winTeamLoseCount=self.homeLoseCount
            self.winTeamDrawCount=self.homeDrawCount

In [16]:
Parser_KBO('20160820','LG')

TypeError: __init__() takes exactly 2 arguments (3 given)